In [5]:
import sys
import regex as re
# !pip install nagisa
import nagisa
import requests
import asyncio as aio
import pandas as pd
API_URL = "https://jisho.org/api/v1/search/words?keyword={}"

In [6]:
class ProgressBar:
    def __init__(self, high, size=50):
        self.high = high
        self.value = 0
        self.size = size

    def __str__(self):
        s = "|"
        s += ((20 * self.value) // self.high) * "█"
        s += ((20 * (self.high - self.value)) // self.high) * " "
        s += "|"
        s += " {}%".format(round(100 * self.value / self.high, 2))

        return s

    def increment(self):
        self.value += 1
        sys.stdout.write("\r{}".format(str(self)))
        sys.stdout.flush()
        

def validate_word(word):
    return (    
            not re.match(r'^\s*$', word)
            and not re.match(r'\W', word)
            and re.match(r'\p{Hiragana}|\p{Katakana}|\p{Han}', word)
            )


def get_meaning(dictionary, word, progress_bar):
    data = requests.get(API_URL.format(word)).json()['data'][0]

    reading = data['japanese'][0]['reading']
    meanings = [x['english_definitions'][0] for x in data['senses']]

    dictionary[word] = {'reading': reading, 'meanings': meanings}

    progress_bar.increment()


def main(meanings,content_string):
    words = set()
    content = content_string
    for word in filter(
        validate_word,
        nagisa.wakati(content)):
        words.add(word)

    print("Extracted {} words".format(len(words)))

    progress_bar = ProgressBar(len(words))
    coroutines = [get_meaning(meanings, word, progress_bar) for word in words]
    aio.wait(coroutines)
    print("\nDone.")

In [8]:
meanings = {}
content_string = "おはようございます。会社に入るときには不織布のマスクを推奨いたします。みなさんご協力をお願いいたします。"
main(meanings,content_string)
i = 1
for word, data in meanings.items():
    print(str(i) + ' ' +(word))
    meaning_df = pd.DataFrame({'Japanese Words':word,'Reading':data['reading'],'meanings':data['meanings']})
    pd.set_option('display.max_rows', len(meaning_df))
    pd.set_option('display.max_columns', None)  
    pd.set_option('display.expand_frame_repr', False)
    #pd.set_option('display.max_colwidth', -1) 
    display(meaning_df)
    i = i+1

Extracted 21 words
|████████████████████| 100.0%
Done.
1 ます


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:53: RuntimeWarning: coroutine 'wait' was never awaited


,Japanese Words,Reading,meanings
0,ます,ます,to increase


2 会社


,Japanese Words,Reading,meanings
0,会社,かいしゃ,company
1,会社,かいしゃ,workplace
2,会社,かいしゃ,Company


...

,Japanese Words,Reading,meanings
0,お,お,tail (of an animal)
1,お,お,"tail (of a kite, etc.)"
2,お,お,tail (of a comet)
3,お,お,slope at the foot of a mountain
4,お,お,O
5,お,お,Tail


20 ご


,Japanese Words,Reading,meanings
0,ご,ご,five


21 みな


,Japanese Words,Reading,meanings
0,みな,みんな,everyone
1,みな,みんな,everything
